<a href="https://colab.research.google.com/github/vinay10949/AnalyticsAndML/blob/master/Python/Text_Encodings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://d2n989decvba0v.cloudfront.net/img/ai-logo-new.png" width="100"/>

## LIVE: Text Encodings
AppliedAICourse


### Agenda
Prerequsites:
- Chapter: "Real world problem: Predict rating given product reviews on Amazon".

Focus:
- Code-walkthrough for each of the methods.
- Data Loading
- Pre-processing using NLTK.
- BoW using SkLearn + Sparse Matrices
- TF-IDF using SkLearn
- Word2VEc using Gensim
- BERT using bert-serving library + APIs.




### Data loading

In [1]:
# We have seen how to mount GDrive in earlier sessions

# This session: download data from internet directly to this box and use it

# Source: http://ai.stanford.edu/~amaas/data/sentiment/
! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2020-06-04 11:06:52--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  18.6MB/s    in 8.9s    

2020-06-04 11:07:01 (9.04 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [0]:
# uncompress and see the data
! ls
! pwd

aclImdb_v1.tar.gz  sample_data
/content


In [2]:
! unzip aclImdb_v1.tar.gz

Archive:  aclImdb_v1.tar.gz
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of aclImdb_v1.tar.gz or
        aclImdb_v1.tar.gz.zip, and cannot find aclImdb_v1.tar.gz.ZIP, period.


In [3]:
#Google: "Unzip tar gz file colab" ----> https://stackoverflow.com/questions/49685924/extract-google-drive-zip-from-google-colab-notebook
import shutil
shutil.unpack_archive("/content/aclImdb_v1.tar.gz", "/content/")
! ls /content/

aclImdb  aclImdb_v1.tar.gz  sample_data


In [4]:
! ls -l /content/aclImdb

total 1724
-rw-r--r-- 1 7297 1000 903029 Jun 11  2011 imdbEr.txt
-rw-r--r-- 1 7297 1000 845980 Apr 12  2011 imdb.vocab
-rw-r--r-- 1 7297 1000   4037 Jun 26  2011 README
drwxr-xr-x 4 7297 1000   4096 Apr 12  2011 test
drwxr-xr-x 5 7297 1000   4096 Jun 26  2011 train


In [6]:
! head -100 /content/aclImdb/imdb.vocab

the
and
a
of
to
is
it
in
i
this
that
was
as
for
with
movie
but
film
on
not
you
he
are
his
have
be
one
!
all
at
by
an
who
they
from
so
like
there
her
or
just
about
if
has
out
what
?
some
good
more
when
she
very
even
my
no
up
time
would
which
only
story
really
their
see
had
can
were
me
we
than
well
much
been
get
people
will
bad
other
also
into
do
because
great
first
how
him
most
don't
its
made
then
them
way
make
could
too
any
after
movies


In [7]:
! ls /content/aclImdb/train


labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [8]:
! ls /content/aclImdb/train/pos

0_9.txt       11876_10.txt  2501_8.txt	 4377_10.txt  625_10.txt   8127_8.txt
10000_8.txt   11877_10.txt  2502_8.txt	 4378_9.txt   6251_7.txt   8128_10.txt
10001_10.txt  11878_10.txt  2503_10.txt  4379_8.txt   6252_10.txt  8129_9.txt
10002_7.txt   11879_10.txt  2504_10.txt  437_9.txt    6253_8.txt   8130_10.txt
10003_8.txt   11880_10.txt  2505_9.txt	 4380_8.txt   6254_10.txt  813_10.txt
10004_8.txt   11881_9.txt   2506_9.txt	 4381_10.txt  6255_8.txt   8131_8.txt
10005_7.txt   11882_8.txt   2507_7.txt	 4382_8.txt   6256_8.txt   8132_10.txt
10006_7.txt   11883_8.txt   250_7.txt	 4383_9.txt   6257_10.txt  8133_7.txt
10007_7.txt   11884_8.txt   2508_10.txt  4384_10.txt  6258_8.txt   8134_10.txt
10008_7.txt   11885_7.txt   2509_9.txt	 4385_9.txt   6259_8.txt   8135_10.txt
1000_8.txt    11886_10.txt  2510_10.txt  4386_9.txt   6260_10.txt  8136_9.txt
10009_9.txt   11887_8.txt   251_10.txt	 4387_9.txt   6261_8.txt   8137_10.txt
10010_7.txt   11888_9.txt   2511_10.txt  4388_9.txt   6262_9.txt   

In [9]:
! cat /content/aclImdb/train/pos/6250_10.txt

There is something special about the Austrian movies not only by Seidl, but by Spielmann and other directors as well. This is the piercing sense of reality that never leaves the viewer throughout the movie. Hundstage is no exception. This effect is achieved not only by the depicted stories but also by actors playing. In Hundstage I have never had the feeling that these are actors playing, but real people instead. So real is the visceral feeling of the viewer...Almost as if the grumpy pensioner or lonely lady in the movie are living below you in your block.<br /><br />Any person living in Vienna can without any doubt painfully recognize the people in the movie with their meckern/sudern (complaining), their hidden sexual urges and the prolo macho guys. This is further reinforced by the Viennese dialect which is, according to many, especially made for complaining as a way of life. A special parochialism and arrogance typical for Vienna are also very well portrayed.<br /><br />The Viennese

In [10]:
# load data from k-reviews from imdb/train/pos reviews
# same code as previous sessions [Dimensionality Reduction session]

k=100

raw_data = [] # empty list, 

# Question: Why list of strings?

index_file = dict(); # store mapping from index to filename

import os
directory = r'/content/aclImdb/train/pos/'

i=0

for f in os.listdir(directory): # for each file in the subfolder
      
  if f.endswith(".txt"): # check for text file
    fname = directory + "/" + f
    
    tmp = open(fname, "r") # read file 

    raw_data.append(tmp.read())
    index_file[i] = fname
        
    i += 1

    if i==k: # read k files
      break


    


print(i)
print(index_file)
print(raw_data[0])



100
{0: '/content/aclImdb/train/pos//9884_10.txt', 1: '/content/aclImdb/train/pos//5489_10.txt', 2: '/content/aclImdb/train/pos//11536_7.txt', 3: '/content/aclImdb/train/pos//8670_8.txt', 4: '/content/aclImdb/train/pos//10356_9.txt', 5: '/content/aclImdb/train/pos//2751_10.txt', 6: '/content/aclImdb/train/pos//3167_7.txt', 7: '/content/aclImdb/train/pos//5539_10.txt', 8: '/content/aclImdb/train/pos//5087_7.txt', 9: '/content/aclImdb/train/pos//10179_9.txt', 10: '/content/aclImdb/train/pos//1957_7.txt', 11: '/content/aclImdb/train/pos//8599_7.txt', 12: '/content/aclImdb/train/pos//1005_10.txt', 13: '/content/aclImdb/train/pos//5012_10.txt', 14: '/content/aclImdb/train/pos//9185_9.txt', 15: '/content/aclImdb/train/pos//10051_10.txt', 16: '/content/aclImdb/train/pos//6944_9.txt', 17: '/content/aclImdb/train/pos//989_9.txt', 18: '/content/aclImdb/train/pos//1435_8.txt', 19: '/content/aclImdb/train/pos//6382_7.txt', 20: '/content/aclImdb/train/pos//143_7.txt', 21: '/content/aclImdb/train/po

### Data preprocessing

In [11]:
# 1. remove stop words + tokenize
# Google "stop word removal python". ----> Many libraries like NLTK, Spacy, Sklearn

# https://pythonprogramming.net/stop-words-nltk-tutorial/

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

example_sent = "This is a sample sentence, showing off the stop words filtration."
def stopWordRemoval(sent, stop_words):
  word_tokens = word_tokenize(sent) # tokenize
  filtered_sentence = "";
  
  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence += w
  return filtered_sentence

stop_words = set(stopwords.words('english')) # NLTK 

print(raw_data[0])
print(stopWordRemoval(raw_data[0], stop_words) ) # call the function


LookupError: ignored

In [16]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

example_sent = "This is a sample sentence, showing off the stop words filtration."
def stopWordRemoval(sent, stop_words):
  word_tokens = word_tokenize(sent) # tokenize
  filtered_sentence = "";
  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence += w
  return filtered_sentence

stop_words = set(stopwords.words('english')) # NLTK 
print(stop_words)

print(raw_data[0])
print(stopWordRemoval(raw_data[0], stop_words) ) # call the function


{'our', 'from', 'your', 'further', 'same', 'aren', 'hers', 'own', 'weren', "shan't", 'we', 'whom', 'o', 'nor', "haven't", 'up', 'not', 'theirs', 'and', 'his', 'don', 'himself', 'because', 'now', 'into', 'all', 'being', 've', 'when', 'through', 'how', 'ma', 'is', 'am', 'some', 'couldn', 'were', 'y', "should've", 'or', 'been', 'myself', 'does', 'during', 'down', 'are', 'few', "you'll", 'very', "won't", 'those', 'did', 'do', 'yourselves', 'any', 'as', 'again', 's', 'these', 'until', 'mustn', 'before', 'then', "hadn't", "wasn't", 'after', 'you', 'its', 'doing', 'while', 'for', 'too', 'should', 'll', 'be', 'they', 're', 'there', 'such', 'their', 'me', 'won', "that'll", 'isn', 'between', "you'd", 'both', 'where', 'the', 'she', "it's", 'ourselves', 'at', 'hasn', 'no', 'can', 'what', 'shouldn', "shouldn't", 'that', 'herself', 'yourself', "didn't", 'them', "she's", 'm', 'ours', "doesn't", "mightn't", 'off', 'd', 'will', 'didn', "don't", "mustn't", 'doesn', 'only', "hasn't", 'itself', "you're", 

In [17]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

example_sent = "This is a sample sentence, showing off the stop words filtration."
def stopWordRemoval(sent, stop_words):
  word_tokens = word_tokenize(sent) # tokenize
  filtered_sentence = "";
  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence += " " + w
  return filtered_sentence

stop_words = set(stopwords.words('english')) # NLTK 
print(stop_words)

print(raw_data[0])
print(stopWordRemoval(raw_data[0], stop_words) ) # call the function


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
{'our', 'from', 'your', 'further', 'same', 'aren', 'hers', 'own', 'weren', "shan't", 'we', 'whom', 'o', 'nor', "haven't", 'up', 'not', 'theirs', 'and', 'his', 'don', 'himself', 'because', 'now', 'into', 'all', 'being', 've', 'when', 'through', 'how', 'ma', 'is', 'am', 'some', 'couldn', 'were', 'y', "should've", 'or', 'been', 'myself', 'does', 'during', 'down', 'are', 'few', "you'll", 'very', "won't", 'those', 'did', 'do', 'yourselves', 'any', 'as', 'again', 's', 'these', 'until', 'mustn', 'before', 'then', "hadn't", "wasn't", 'after', 'you', 'its', 'doing', 'while', 'for', 'too', 'should', 'll', 'be', 'they', 're', 'there', 'such', 'their', 'me', 'won', "that'll", 'isn', 'between', "you'd", 'both', 'where', 'the', 'she', "it's", 'ourselves', 'at', 'hasn', 'no'

In [18]:
# Lemmatize
# Source: https://pythonprogramming.net/lemmatizing-nltk-tutorial/

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

print(lemmatizer.lemmatize("cats"))
print(lemmatizer.lemmatize("cacti"))
print(lemmatizer.lemmatize("geese"))
print(lemmatizer.lemmatize("rocks"))
print(lemmatizer.lemmatize("python"))
print(lemmatizer.lemmatize("better", pos="a"))
print(lemmatizer.lemmatize("best", pos="a"))
print(lemmatizer.lemmatize("run"))
print(lemmatizer.lemmatize("run",'v'))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
cat
cactus
goose
rock
python
good
best
run
run


In [19]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

example_sent = "This is a sample sentence, showing off the stop words filtration."
def stopWord_Lemmatize(sent, stop_words, lemmatizer ):
  word_tokens = word_tokenize(sent) # tokenize
  return_sent = "";
  
  for w in word_tokens:
      if w not in stop_words:
          return_sent += lemmatizer.lemmatize(w) # lemmatize w beofre adding it to the return_sent
  return return_sent

stop_words = set(stopwords.words('english')) # NLTK 
lemmatizer = WordNetLemmatizer()

print(raw_data[0])
print(stopWord_Lemmatize(raw_data[0], stop_words, lemmatizer) ) # call the function


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Rawhide was a wonderful TV western series. Focusing on a band of trail drovers lead by the trail boss Gil Favor. Most episodes - especially from the first 3 seasons were really character studies of Favor and his men. Guest stars came and went but unlike Wagon Train they seldom dominated the episodes they appeared in. Rawhide was a true, gritty western and Gil Favor stood out as a memorable character never to be forgotten. Thanks to Eric Fleming's performance the show became a massive hit. Of course he was ably supported by a wonderful cast of good actors - Clint Eastwood, Sheb Wooley, Paul Brinegar, Steve Raines, James Murdoch, Rocky Shahan, Robert Cabal. All of these actors left their mark in a piece of television history. Rawhide captured the flavour of that

In [0]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

example_sent = "This is a sample sentence, showing off the stop words filtration."
def stopWord_Lemmatize(sent, stop_words, lemmatizer ):
  word_tokens = word_tokenize(sent) # tokenize
  return_sent = "";
  
  for w in word_tokens:
      if w not in stop_words:
          return_sent += " " + lemmatizer.lemmatize(w) # lemmatize w beofre adding it to the return_sent
  return return_sent

stop_words = set(stopwords.words('english')) # NLTK 
lemmatizer = WordNetLemmatizer()

print(raw_data[0])
print(stopWord_Lemmatize(raw_data[0], stop_words, lemmatizer) ) # call the function


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
What's not to like about this movie? Every year you know that you're going to get one or two yule tide movies during Christmas time and most of them are going to be terrible. This movie is definitely a fresh new idea that was pulled off pretty well. A very funny take on a rich young guy paying a family to simulate a real Christmas for him. What is the good of having money like that if you can't do fun things with it. It was a win-win situation. A regular family gets six figures and a rich guy gets to experience Christmas like he imagined. Only if.<br /><br />Drew Latham (Ben Affleck) was incredibly difficult to deal with and it was just a riot to see the family reluctantly comply with his absurd demands. It was a fun and funny movie.
 What 's like movie ? Ever

In [0]:

# pre-process the k sentences and store the result
processed_data = []

for sent in raw_data:
  processed_data.append(stopWord_Lemmatize(sent, stop_words, lemmatizer) )

### Bag-of-words (BoW)

In [21]:
# Google "sklearn bag of words" ---> https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer() #https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
X_BoW = count_vect.fit_transform(processed_data)

print(count_vect.get_feature_names()) 

['000bet', '000lifefirstday', '10', '1010', '10itemsless', '10three', '14tharrondissement', '1950', '1950life', '1951pres', '1969', '1986', '1995', '1996', '1997', '2004remake', '2006', '2007', '29', '301950', '51', '51koreanwargen', '5saydefinitelyworthlook', '6ratingimdbjohnnydangerouslymomentiwrotereviewithoughtisomethingpromotememorablepiececomedymuchi', '70', '73', '750milliondollarworth', '80', '82minutetime', 'aaverageidiotamericanmoviegoern', 'ablackcatclimbfireescapeentersapartment', 'abouteuropa', 'absolutelyincredible', 'absorbedrealparent', 'absorbing', 'academyfollowedsuitgivingday', 'acarefullychosencastincreaseproceedingfinestyle', 'accept', 'acclaimedfrenchfilm', 'accordingextremelyrichfather', 'accordingnarrator', 'acrossnorthkoreanborderevenabandoningsouthkoreancapitalcityseoul', 'act', 'acteddeepfeelingintenseconcentration', 'actednicelydirected', 'actingdefinitelytoplist', 'actionalwaysexcitinglyphotographed', 'actionflick', 'actorendsituationpreviousfilm', 'actorha

In [22]:
print(type(X_BoW))

<class 'scipy.sparse.csr.csr_matrix'>


In [23]:
# Sparse representations vs dense Matrices

#Refer: https://docs.scipy.org/doc/scipy/reference/sparse.html


print(X_BoW.data.nbytes) # Refer: https://stackoverflow.com/questions/43681279/why-is-scipy-sparse-matrix-memory-usage-indifferent-of-the-number-of-elements-in

# Convert X_BoW to dense
# Ref: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.todense.html#scipy.sparse.csr_matrix.todense
X_BoW_Dense = X_BoW.todense();
print(X_BoW_Dense.data.nbytes)

print(X_BoW.shape)
print(X_BoW_Dense.shape)
# Always use sparse-matrix in these situations

28632
2682400
(100, 3353)
(100, 3353)


In [0]:
print(X_BoW[0,:])

  (0, 4264)	2
  (0, 2271)	3
  (0, 2539)	4
  (0, 1341)	1
  (0, 4357)	1
  (0, 2165)	1
  (0, 3093)	1
  (0, 1698)	2
  (0, 1670)	3
  (0, 2717)	1
  (0, 4047)	1
  (0, 4368)	1
  (0, 3937)	1
  (0, 716)	3
  (0, 3942)	1
  (0, 3875)	1
  (0, 3916)	1
  (0, 1025)	1
  (0, 1607)	1
  (0, 2613)	1
  (0, 1923)	1
  (0, 3039)	1
  (0, 2978)	1
  (0, 4257)	1
  (0, 1630)	2
  :	:
  (0, 3910)	1
  (0, 2054)	2
  (0, 4296)	2
  (0, 3536)	1
  (0, 3152)	1
  (0, 3537)	1
  (0, 1497)	1
  (0, 1378)	1
  (0, 1936)	1
  (0, 2720)	1
  (0, 1929)	1
  (0, 517)	2
  (0, 1187)	1
  (0, 2205)	1
  (0, 426)	1
  (0, 135)	1
  (0, 1964)	1
  (0, 1090)	1
  (0, 1002)	1
  (0, 3254)	1
  (0, 3389)	1
  (0, 3170)	1
  (0, 818)	1
  (0, 68)	1
  (0, 1034)	1


In [0]:
print(X_BoW[0,818])
print(X_BoW[0,817])

1
0


In [0]:
print(X_BoW_Dense[0,:])
print(X_BoW_Dense[0,818])
print(X_BoW_Dense[0,817])

[[0 0 0 ... 0 0 0]]
1
0


### TD-IDF

In [24]:
# Google: "TF-IDF SkLearn" ---> https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
X_tfidf = vectorizer.fit_transform(raw_data)

print(vectorizer.get_feature_names())

['000', '000 bet', '000 lives', '10', '10 10', '10 agree', '10 br', '10 grade', '10 idiots', '10 important', '10 items', '10 minute', '10 minutes', '10 movie', '10 saw', '100', '100 casualties', '100 wonderfully', '11', '11 1951', '13', '13 boyfriends', '14th', '14th arrondissement', '15', '15 1996', '15 feet', '16éme', '16éme catalina', '17', '17 2006', '1876', '1876 stewart', '19', '19 years', '1920', '1920 year', '1930s', '1930s supporting', '1937', '1937 film', '1937 incident', '1939', '1939 charlie', '1940s', '1940s period', '1942', '1942 br', '1950', '1950 51', '1950 communist', '1950 lives', '1951', '1951 pres', '1952', '1952 polls', '1960', '1960 television', '1962', '1962 gen', '1962 honor', '1965', '1965 cbs', '1969', '1969 great', '1970', '1970 80', '1978', '1978 film', '1980', '1980 disappearance', '1980 emotionally', '1980s', '1980s br', '1981', '1981 film', '1983', '1983 entry', '1986', '1986 rate', '1989', '1989 10', '1989 year', '1991', '1991 nbc', '1995', '1995 video',

In [25]:
print(type(X_tfidf))
print(X_tfidf.shape)
print(X_tfidf.data.nbytes) # Refer: https://stackoverflow.com/questions/43681279/why-is-scipy-sparse-matrix-memory-usage-indifferent-of-the-number-of-elements-in

X_tfidf_dense = X_tfidf.todense()
print(type(X_tfidf_dense))
print(X_tfidf_dense.shape)
print(X_tfidf_dense.data.nbytes)

<class 'scipy.sparse.csr.csr_matrix'>
(100, 15374)
162904
<class 'numpy.matrix'>
(100, 15374)
12299200


### Word2Vec

In [26]:
# Gensim
# Refer: https://machinelearningmastery.com/develop-word-embeddings-python-gensim/

# Download pretrained  vectors
# https://www.quora.com/How-can-I-download-the-Google-news-word2vec-pretrained-model-from-a-Ubuntu-terminal
! wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
! gunzip GoogleNews-vectors-negative300.bin.gz

--2020-06-04 13:08:09--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.28.38
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.28.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  31.9MB/s    in 50s     

2020-06-04 13:09:00 (31.3 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [0]:
! ls

GoogleNews-vectors-negative300.bin  sample_data


In [27]:
# takes time as the model is large to laod into RAM
# RAM consumption also shoots up. Kernel could restart and give you a larger RAM isnatnce like 25GB RAM

from gensim.models import KeyedVectors
filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [28]:
result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('queen', 0.7118192911148071)]


In [29]:
v = model['queen'];
print(type(v))
print(v.shape)

<class 'numpy.ndarray'>
(300,)


### BERT

- BERT vs Word2Vec
- BERT: Contextual encodings

In [0]:
# There are many ways to obatin BERT encodings. We are using one of the 
# most simple+popular approaches

# In DL chpaters, we will see how to do it in Keras.

# use GPU based instance: Runtime---> Change Runtime --> GPU

# Chaneg to TensorFlow Version 1.x 
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [30]:
#https://github.com/hanxiao/bert-as-service

# https://github.com/hanxiao/bert-as-service/issues/380
# Install BERT-SERVING client and Server
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

     |████████████████████████████████| 71kB 4.8MB/s 
     |████████████████████████████████| 358kB 14.9MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=6d9ac2b0512d3335ad0dde1c1f83cfbdd305b3a277eed3d8868b2732be77ab91
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
  Created wheel for flask-compress: filename=Flask_Compress-1.5.0-cp36-none-any.whl size=5273 sha256=8a1893da7bdee203a1f35934b8a156864854edba476749e7475964d742795b5e
  Stored in directory: /root/.cache/pip/wheels/f7/e9/e4/5afc286be7c87461375e33152558415dfeb0c8f5af3b50e742
Successfully built GPUtil flask-compress


In [31]:
# dowload pretrained models
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2020-06-04 13:29:05--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.218.128, 2a00:1450:4013:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.218.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  86.4MB/s    in 4.5s    

2020-06-04 13:29:10 (86.4 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [0]:
# Start BERT_SERVER on the current computer
!nohup bert-serving-start -model_dir=./uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [0]:
# Use bert-client from python
# Takes time to execute as it uses GPU
from bert_serving.client import BertClient
bc = BertClient()
print (bc.encode(['First do it', 'then do it right', 'then do it better'])) # list of setences

In [0]:
v = bc.encode([raw_data[0]])

In [0]:
print(type(v))
print(v.shape)